In [2]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using DifferentialEquations
using MeshIO

using VMRobotControl: remake
include("../functions.jl")

display_transform (generic function with 1 method)

## Creating a simple mechanism for test purposes

In [3]:
mechanism = Mechanism{Float64}("TestRobot")
F0 = root_frame(mechanism)
F1 = add_frame!(mechanism; id="L1_frame")

# Define the unit vectors to make things easier
X = SVector(1., 0., 0.)
Y = SVector(0., 1., 0.)
Z = SVector(0., 0., 1.)

J1 = Prismatic(X)
add_joint!(mechanism, J1; parent="root_frame", child="L1_frame", id="J1")

"J1"

In [4]:
display(mechanism)
display(frames(mechanism))
display(joints(mechanism))

1DOF Mechanism{Float64} "TestRobot" with 2 frames, 1 joint

2-element Vector{String}:
 "root_frame"
 "L1_frame"

OrderedCollections.OrderedDict{String, VMRobotControl.MechanismJoint{<:VMRobotControl.AbstractJointData{Float64}, String}} with 1 entry:
  "J1" => MechanismJoint{PrismaticData{Float64}, String}(PrismaticData{Float64}…

In [4]:
add_coordinate!(mechanism, FrameOrigin(F0); id="base_centre_of_mass")
add_coordinate!(mechanism, FrameOrigin(F1); id="ee_centre_of_mass")

add_component!(mechanism, PointMass(1.0, "base_centre_of_mass"); id="base_mass")
add_component!(mechanism, PointMass(1.0, "ee_centre_of_mass"); id="ee_mass")

I_mat = @SMatrix [
    0.01  0.    0.  ;
    0.    0.01  0.  ;
    0.    0.    0.01
]

add_inertia!(mechanism, F1, I_mat; id="L1_inertia")

1DOF Mechanism{Float64} "TestRobot" with 2 frames, 1 joint, 3 coordinates, 3 components

In [5]:
add_coordinate!(mechanism, JointSubspace("J1"); id="J1")
add_component!(mechanism, LinearDamper(0.5, "J1"); id="J1_damper")
add_gravity_compensation!(mechanism, VMRobotControl.DEFAULT_GRAVITY)

## Vizualizing the robot

In [7]:
m = compile(mechanism)
kcache = Observable(new_kinematics_cache(m))

# Setup the figure
fig1 = Figure(size=(700, 350))
ls = LScene(fig1[1, 1]; show_axis=false)

# Or a sketch of it's kinematic structure
robotsketch!(ls, kcache; scale=0.3, linewidth=2.5, transparency=true)

display(fig1)


GLMakie.Screen(...)

## Introducing virtual mechanisms

In [7]:
vms = VirtualMechanismSystem("testVMS", mechanism)
root = root_frame(vms.robot)

refpos = -0.5*X 
add_coordinate!(vms, FrameOrigin(".robot.L1_frame"); id="ee_position")
add_coordinate!(vms, ConstCoord(refpos); id="ref")
add_coordinate!(vms, CoordDifference("ee_position", "ref"); id="ee_error")
add_coordinate!(vms, FramePoint(".robot.$root", SVector(0.0, -0.075, 0.32));  id="Target position")

ee_stiffness = SMatrix{3, 3}(100.0, 0.0, 0.0, 0.0, 100.0, 0.0, 0.0, 0.0, 100.0)
ee_damping = SMatrix{3, 3}(15.0, 0.0, 0.0, 0.0, 15.0, 0.0, 0.0, 0.0, 15.0)

add_component!(vms, LinearSpring(ee_stiffness, "ee_error"); id="ee_spring")
add_component!(vms, LinearDamper(ee_damping, "ee_error"); id="ee_damper")
add_component!(vms, ForceSource(SVector(60.0),30.0, ".robot.J1"); id="force source")

"force source"

## Simulating

In [10]:
vms_compiled = compile(vms)
dcache = new_dynamics_cache(vms_compiled)
force_source_component_id = get_compiled_componentID(dcache, "force source")
struct_fields_and_types(typeof(vms_compiled[force_source_component_id]))

force_max::SVector{1, Float64}
power_max::Float64
coord::VMSCoordID{JointSubspace{VMRobotControl.CompiledJointID{CompiledMechanismJoint(Float64, PrismaticData{Float64})}}, robot_coord}


In [40]:
phase_2 = false

f_setup(cache) = return (get_compiled_coordID(cache, "ref"), phase_2, get_compiled_componentID(cache, "ee_spring"))

function f_control(cache, t, args, extra)
    ref_pos_coord_id, phase_2, ee_spring_component_id = args
    
    if t > 3 && !phase_2
        cache[ref_pos_coord_id] = remake(cache[ref_pos_coord_id]; coord_data=ConstCoord(-1*X))
        phase_2 = true
    end 

    if t > 6 
        new_stiffness = SMatrix{3, 3}(1000.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 1000.0) 
        cache[ee_spring_component_id] = remake(cache[ee_spring_component_id] ; stiffness = new_stiffness)
    end

    nothing
end

tspan = (0., 10.)
vms_compiled = compile(vms)
q = (zero_q(vms_compiled.robot), Float64[]) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), Float64[]) # Robot joint velocity, vm joint velocities
g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

ODE   0%|█                                              |  ETA: N/A
ODE 100%|███████████████████████████████████████████████| Time: 0:00:00


## Visualizing the result

In [41]:
dcache = new_dynamics_cache(m)
display(fig1)
animate_robot_odesolution(fig1, sol, kcache, "robot_animation.mp4")

"robot_animation.mp4"